In [9]:
import pandas as pd
child_mortality = pd.read_csv('child-mortality.csv')
child_mortality.head(100)

,Entity,Code,Year,"Child mortality (Select Gapminder, v10) (2017)"
0,Afghanistan,AFG,1957,381.194
1,Afghanistan,AFG,1958,375.190
2,Afghanistan,AFG,1959,369.361
3,Afghanistan,AFG,1960,363.700
4,Afghanistan,AFG,1961,357.500
...,...,...,...,...
95,Albania,ALB,2013,14.900
96,Albania,ALB,2014,14.400
97,Albania,ALB,2015,14.000
98,Albania,ALB,2016,13.500


In [10]:
child_mortality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13512 entries, 0 to 13511
Data columns (total 4 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Entity                                          13512 non-null  object 
 1   Code                                            13512 non-null  object 
 2   Year                                            13512 non-null  int64  
 3   Child mortality (Select Gapminder, v10) (2017)  13512 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 422.4+ KB


### TASK 1 : Minimum values of child mortality in different countries

In [11]:
child_mortality.rename(columns={"Child mortality (Select Gapminder, v10) (2017)": "Child_mortality"}, inplace=True)
df = child_mortality[["Entity", "Code", "Child_mortality"]]
df.groupby("Code").min().sort_values("Child_mortality")

,Entity,Child_mortality
Code,,
ISL,Iceland,2.100
HKG,Hong Kong,2.282
FIN,Finland,2.300
SVN,Slovenia,2.300
LUX,Luxembourg,2.400
...,...,...
MLI,Mali,110.600
SLE,Sierra Leone,113.500
CAF,Central African Republic,123.600


### TASK 2 : Least value of CM in countries in 2016

In [12]:
# Check if 2017 is available for all countries
df = child_mortality[child_mortality["Year"] == 2016]
df = df[["Entity", "Code", "Child_mortality"]]  # Use only relevant columns
df.sort_values("Child_mortality", inplace=True)

df.head()

,Entity,Code,Child_mortality
5528,Iceland,ISL,2.100
5217,Hong Kong,HKG,2.282
10883,Slovenia,SVN,2.300
4087,Finland,FIN,2.300
7229,Luxembourg,LUX,2.400


### TASK 3 : Plot this on the map. Create a Choropleth

In [13]:
import folium
from folium import Choropleth
import geopandas as gpd

In [14]:
worldfilepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(worldfilepath)
print(len(world))
world.head()

177


,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [15]:
child_mortality.rename(columns={"Code": "iso_a3"}, inplace=True)
child_mortality.head()

,Entity,iso_a3,Year,Child_mortality
0,Afghanistan,AFG,1957,381.194
1,Afghanistan,AFG,1958,375.190
2,Afghanistan,AFG,1959,369.361
3,Afghanistan,AFG,1960,363.700
4,Afghanistan,AFG,1961,357.500


In [27]:
df = child_mortality[child_mortality["Year"] == 2016]
df = df.merge(world, on="iso_a3")

c_m = df[["iso_a3", "Child_mortality"]].set_index("iso_a3")
country_geometry = df[["iso_a3", "geometry"]].set_index("iso_a3")

country_geom_gdf = gpd.GeoDataFrame(country_geometry, crs="EPSG:4326")
country_geom_gdf.head()

,geometry
iso_a3,
AFG,"POLYGON ((66.51861 37.36278, 67.07578 37.35614..."
ALB,"POLYGON ((21.02004 40.84273, 20.99999 40.58000..."
DZA,"POLYGON ((-8.68440 27.39574, -8.66512 27.58948..."
AGO,"MULTIPOLYGON (((12.99552 -4.78110, 12.63161 -4..."
ARG,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000..."


In [39]:
# Finally, lets create the map
m = folium.Map(location=[0.0, 0.0], tiles='openstreetmap', zoom_start=2)

Choropleth(geo_data=country_geom_gdf.__geo_interface__,
          data=c_m["Child_mortality"], key_on='feature.id', fill_color='YlGnBu', legend_name='Child Mortality(2016)').add_to(m)
m.save('Child_mortality-2016.html')
m

### TASK 4 : Compare this to how it was back in 1990

In [38]:
# Get data for 1990
data_1990 = child_mortality[child_mortality["Year"] == 1990].copy()

# Merge data to make sure only countries for which data is available are plotted
merged_df = data_1990.merge(world, on="iso_a3")

# Convert to GeoDataFrame
geom_1990 = gpd.GeoDataFrame(merged_df[["iso_a3", "geometry"]].set_index("iso_a3"), crs="EPSG:4326")

# 1990's data for child_mortality
data = merged_df[["iso_a3", "Child_mortality"]].set_index("iso_a3")

In [42]:
m = folium.Map(location=[0.0, 0.0], tiles='openstreetmap', zoom_start=2)
Choropleth(geo_data=geom_1990.__geo_interface__, data=data["Child_mortality"],
          key_on='feature.id', fill_color='YlGnBu', legend_name="Child Mortality(1990)").add_to(m)

m.save('Child_mortality-1990.html')
m